In [1]:
from Bidimensional_protein_folding import ProteinEnv
import collections
import matplotlib.pyplot as plt
import numpy as np
import random
from copy import deepcopy


In [2]:
Q_dict = collections.defaultdict(dict)

print(len(Q_dict))

0


In [3]:

def add_to_dict(state):
    if (state[0], tuple(state[1])) not in Q_dict.keys():
        Q_dict[state[0], tuple(state[1])] = {}
        for action in range(4):
            Q_dict[state[0], tuple(state[1])][action] = 0.0

In [4]:
print(Q_dict.keys())

dict_keys([])


In [5]:
max_epsilon = 1.0
min_epsilon = 0.1
decay_rate = 0.000001

def epsilon_greedy(state, time):
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*time)
    #print('epsilon ', epsilon)
    z = np.random.random()
    #print('z ', z)
        
    if z > epsilon:
        action = max(Q_dict[state[0], tuple(state[1])],key=
                     Q_dict[state[0], tuple(state[1])].get)   #Exploitation: this gets the action corresponding to max q-value of current state
    else:
        action = np.random.choice(np.arange(0,4))    #Exploration: randomly choosing and action
    
    return action

In [ ]:
def epsilon_greedy(state,time):
    e = 0
    if np.random.rand(1) < e:
        action = np.random.choice(np.arange(0,4))    #Exploration: randomly choosing and action
    else:
        action = max(Q_dict[state[0], tuple(state[1])],key=
                     Q_dict[state[0], tuple(state[1])].get)
    return action

### Training

In [6]:
#Defining parameters for the experiment

EPISODES = 10
LR = 0.0001                   #learning rate
GAMMA = 0.9

In [7]:
for episode in range(0,EPISODES):
    if episode % 100000 == 0:
        print('''
    ##################################
        # episode ''',episode,'''
    ###################################''')
    env = ProteinEnv()      #creating an instance of the class
    #print('protein ', env.protein)
    curr_state = env.state
    add_to_dict(curr_state)
    reward = None    
    
    while len(curr_state[1])<len(curr_state[0]):  
        
        #print('current state [1] in while loop', curr_state[1])
        #print('size state[1]=',len(curr_state[1]),' size state[0]=',len(curr_state[0]))
        
        curr_action = epsilon_greedy(curr_state, episode)
        
        next_state, reward = env.step(curr_state, curr_action)
        #print('reward ', reward)
        
        add_to_dict(next_state)

        # UPDATE RULE
        
        max_next = max(Q_dict[next_state[0], tuple(next_state[1])],key=Q_dict[next_state[0], tuple(next_state[1])].get)   #this gets the action corresponding to max q-value of next state

        
         
        #print('Q_dict before', Q_dict[curr_state[0], tuple(curr_state[1])][curr_action])
        
        #without learning rate
        Q_dict[curr_state[0], tuple(curr_state[1])][curr_action] =\
        reward + GAMMA*(Q_dict[next_state[0], tuple(next_state[1])][max_next]) 
        
        #with learning rate
        #Q_dict[curr_state[0], tuple(curr_state[1])][curr_action] =\
        #(1-LR)*Q_dict[curr_state[0], tuple(curr_state[1])][curr_action]+ LR *(reward + (GAMMA*(Q_dict[next_state[0], tuple(next_state[1])][max_next])))  


        #print('Q_dict after', Q_dict[curr_state[0], tuple(curr_state[1])][curr_action])
        
        curr_state = next_state
        #print(next_state)



    ##################################
        # episode  0 
    ###################################


In [13]:
###### len(Q_dict)
next_state=('HHPHHHPHH', [(0, 0), (0, -1), (0, 0)] )
#max_next = max(Q_dict[next_state[0], tuple(next_state[1])],key=Q_dict[next_state[0], tuple(next_state[1])].get)
print(curr_state[0])
print(next_state[0])

print(tuple(curr_state[1]))
print(tuple(next_state[1]))

print(curr_state[1])
print(next_state[1])

print(Q_dict[next_state[0], tuple(next_state[1])].get(1))

ma = max(Q_dict[curr_state[0], tuple(curr_state[1])],key=
                       Q_dict[curr_state[0], tuple(curr_state[1])].get) 
ma



HHPHHHPHH
HHPHHHPHH
((0, 0), (0, 1), (1, 1), (1, 2), (1, 1), (1, 2), (1, 1), (2, 1), (3, 1))
((0, 0), (0, -1), (0, 0))
[(0, 0), (0, 1), (1, 1), (1, 2), (1, 1), (1, 2), (1, 1), (2, 1), (3, 1)]
[(0, 0), (0, -1), (0, 0)]
None


0

In [ ]:
Q_dict

In [ ]:
#Defining a function which will take as input the protein and return the arrangement/policy

def policy(protein):
    curr_state = (protein, [(0, 0)])
    
    while len(curr_state[1])<len(curr_state[0]):    
        best_action = max(Q_dict[curr_state[0], tuple(curr_state[1])],key=Q_dict[curr_state[0], tuple(curr_state[1])].get)
        next_state = env.transition(curr_state, best_action)
        print('Q_dict after', Q_dict[curr_state[0], tuple(curr_state[1])][curr_action])
        curr_state = next_state
        #print('Q_dict after', Q_dict[curr_state[0], tuple(curr_state[1])][curr_action])

    
    x_h_coords = []
    y_h_coords = []
    x_p_coords = []
    y_p_coords = []
    
    x_inter_coords = []
    y_inter_coords = []
    

    for ix in range(len(next_state[0])):
        if next_state[0][ix] == 'H':
            x_h_coords.append(next_state[1][ix][0])
            y_h_coords.append(next_state[1][ix][1])
        if next_state[0][ix] == 'P':
            x_p_coords.append(next_state[1][ix][0])
            y_p_coords.append(next_state[1][ix][1])
        
       #inter H-P
    for ix in range(len(next_state[0])-1):
        x_inter_coords.append(((next_state[1][ix][0])+(next_state[1][ix+1][0]))/2)
        y_inter_coords.append(((next_state[1][ix][1])+(next_state[1][ix+1][1]))/2)
            
        
        
    
    plt.scatter(x_h_coords, y_h_coords, marker = 'o', color = 'black')
    plt.scatter(x_p_coords, y_p_coords, marker = 'o', facecolors='none', edgecolors='black')
    plt.scatter(x_inter_coords, y_inter_coords, marker = 'o', color = 'red')

    plt.show()

    
    
    
    print(x_h_coords)
    print(y_h_coords)
    print(x_p_coords)
    print(y_p_coords)
    print(x_inter_coords)
    print(y_inter_coords)

### Check Policy

In [ ]:
policy('HHPHHH')
